In [4]:
import xml.etree.ElementTree as ET
import pandas as pd
import random
from datetime import datetime, timedelta

# Paths to your XML files
trip_file = r'C:\Users\abhi1\Downloads\sumo-example-main\sumo-example-main\2021-05-01-22-25-37\osm.passenger.trips.xml'
type_file = r'C:\Users\abhi1\Downloads\sumo-example-main\sumo-example-main\2021-05-01-22-25-37\osm.net.xml'

# Parse trip data
trip_tree = ET.parse(trip_file)
trip_root = trip_tree.getroot()

# Parse type data
type_tree = ET.parse(type_file)
type_root = type_tree.getroot()

# Get speed values by type from type XML
speed_dict = {}
for elem in type_root.findall('type'):
    type_id = elem.get('id')
    speed = float(elem.get('speed')) if elem.get('speed') else 0.0
    speed_dict[type_id] = speed

# Extract trips and calculate fields
data = []
start_time = datetime.strptime("2024-11-02 00:00:01", "%Y-%m-%d %H:%M:%S")
for i, trip in enumerate(trip_root.findall('trip')):
    timestamp = start_time + timedelta(seconds=i)
    
    # Get trip attributes
    vehicle_type = trip.get('type')
    speed = speed_dict.get(vehicle_type, 0.0) + random.randint(-5, 5)  # add variability
    
    # Random values for example fields
    vehicle_count = random.randint(5, 25)
    distance = random.randint(40, 130)
    traffic_density = round(random.uniform(0.2, 1.0), 2)
    
    # Append row
    data.append([timestamp, speed, vehicle_count, distance, traffic_density])

# Create DataFrame and write to CSV
df = pd.DataFrame(data, columns=['timestamp', 'speed', 'vehicle_count', 'distance', 'traffic_density'])
df.to_csv(r'C:\Users\abhi1\Downloads\sumo-example-main\sumo-example-main\Traffic_data_maker\traffic_data.csv', index=False)

print("Data successfully saved to traffic_data.csv")


Data successfully saved to traffic_data.csv


In [5]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.utils import class_weight

# Load and preprocess your dataset
data = pd.read_csv(r'C:\Users\abhi1\Downloads\sumo-example-main\sumo-example-main\Traffic_data_maker\traffic_data.csv')  # Replace with your dataset file

# Features and target with additional feature engineering
data['congestion_level'] = data['vehicle_count'] / (data['speed'] + 1e-5)  # Avoid division by zero
X = data[['speed', 'vehicle_count', 'distance', 'congestion_level']]
y = data['traffic_density']

# Normalize features
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Define a threshold for binary classification
threshold = y_train.mean()  # Adjust threshold if needed
y_train_binary = (y_train > threshold).astype(int)
y_test_binary = (y_test > threshold).astype(int)

# Calculate class weights to handle imbalance
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_train_binary), y=y_train_binary)
class_weights_dict = {i: class_weights[i] for i in range(len(class_weights))}

# Build a new model architecture
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile model with class weights
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model with early stopping and class weights
model.fit(X_train, y_train_binary, epochs=30, batch_size=16, validation_split=0.2, class_weight=class_weights_dict, callbacks=[early_stopping])

# Predict on the test set
y_pred = model.predict(X_test).flatten()
y_pred_binary = (y_pred > 0.5).astype(int)

# Calculate metrics
accuracy = accuracy_score(y_test_binary, y_pred_binary)
precision = precision_score(y_test_binary, y_pred_binary)
recall = recall_score(y_test_binary, y_pred_binary)
f1 = f1_score(y_test_binary, y_pred_binary)

print(f'Adjusted Accuracy: {accuracy:.4f}')
print(f'Adjusted Precision: {precision:.4f}')
print(f'Adjusted Recall: {recall:.4f}')
print(f'Adjusted F1 Score: {f1:.4f}')

# Save the updated model
model.save('adjusted_traffic_density_model.h5')


Epoch 1/30
360/360 [==============================] - 1s 2ms/step - loss: 0.6938 - accuracy: 0.5042 - val_loss: 0.6942 - val_accuracy: 0.5076
Epoch 2/30
360/360 [==============================] - 1s 2ms/step - loss: 0.6928 - accuracy: 0.5111 - val_loss: 0.6933 - val_accuracy: 0.5014
Epoch 3/30
360/360 [==============================] - 1s 2ms/step - loss: 0.6940 - accuracy: 0.4991 - val_loss: 0.6937 - val_accuracy: 0.4938
Epoch 4/30
360/360 [==============================] - 1s 2ms/step - loss: 0.6931 - accuracy: 0.5125 - val_loss: 0.6938 - val_accuracy: 0.5056
Epoch 5/30
360/360 [==============================] - 1s 2ms/step - loss: 0.6934 - accuracy: 0.5052 - val_loss: 0.6937 - val_accuracy: 0.5125
Epoch 6/30
360/360 [==============================] - 1s 2ms/step - loss: 0.6930 - accuracy: 0.5063 - val_loss: 0.6939 - val_accuracy: 0.5000
Epoch 7/30
57/57 [==============================] - 0s 1ms/step
Adjusted Accuracy: 0.5172
Adjusted Precision: 0.5248
Adjusted Recall: 0.3073
Adjuste

c:\Users\abhi1\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
